In [21]:
import List_to_Pytorch_Dataset
import importlib
import torch
from torch import nn
from torch.utils.data import DataLoader
import CNN1D
import Smaller_input_CNN
import Neural_Networks
from statistics import mean
importlib.reload(List_to_Pytorch_Dataset)
importlib.reload(DeepCNN)

<module 'DeepCNN' from 'd:\\4IF\\Stage\\Projet_stage\\fall-detection-pytorch\\DeepCNN.py'>

In [5]:
print(torch.__version__)

2.0.1


In [6]:
dataset = List_to_Pytorch_Dataset.MobifallData(augment=False)

STD 180
WAL 183
JOG 183
JUM 183
STU 200
STN 200
SCH 200
SIT 190
CHU 114
CSI 200
CSO 197
FOL 192
FKL 192
BSC 191
SDL 192


In [7]:
print(f"Dataset size: {len(dataset)}")
print(f"Data sample size: {len(dataset[0][0][0])}")

Dataset size: 2797
Data sample size: 250


In [20]:
print(len(dataset[0]))

2


In [8]:
device = 'cpu' #cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
torch.cuda.empty_cache()

Using cpu device


In [10]:
labels = { 0  : "STD",
           1  : "WAL",
           2  : "JOG",
           3  : "JUM",
           4  : "STU",
           5  : "STN",
           6  : "SCH",
           7  : "SIT",
           8  : "CHU",
           9  : "CSI",
           10 : "CSO",
           11 : "FOL",
           12 : "FKL",
           13 : "BSC",
           14 : "SDL"
           #15 : "PFF"
           }

classification_map = {}
classification_map["Falls"] = {"TP" : 0,
                                 "FP" : 0,
                                 "TN" : 0,
                                 "FN" : 0}
for key, value in labels.items():
    classification_map[value] = {"TP" : 0,
                                 "FP" : 0,
                                 "TN" : 0,
                                 "FN" : 0}

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [12]:
def is_fall(label):
    return label in [11, 12, 13, 14]

In [13]:
def valid_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    valid_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X)

            for key, value in labels.items():
                for i in range(len(y)):
                    if y[i] == key:
                        if pred[i].argmax() == key:
                            classification_map[value]["TP"] += 1
                        else:
                            classification_map[value]["FN"] += 1
                    elif pred[i].argmax() == key:
                        classification_map[value]["FP"] += 1
                    else:
                        classification_map[value]["TN"] += 1
                        
            #this is for fall/not-fall classification
            for i in range(len(y)):
                if is_fall(y[i]):
                    if is_fall(pred[i].argmax()):
                        classification_map["Falls"]["TP"] += 1
                    else:
                        classification_map["Falls"]["FN"] += 1
                elif is_fall(pred[i].argmax()):
                    classification_map["Falls"]["FP"] += 1
                else:
                    classification_map["Falls"]["TN"] += 1
                    

            #model loss and accuracy
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    #model stats
    valid_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {valid_loss:>8f} \n\n")
    return 100*correct

In [22]:
def main():
    model = DeepCNN.CNN().to(device)
    model.train()

    #this is for mobile use adaptation
    #backend = "qnnpack"
    #model.qconfig = torch.quantization.get_default_qat_qconfig(backend)
    #model = torch.quantization.prepare_qat(model, inplace=False)

    learning_rate = 0.001
    epochs = 40

    # Initialize the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    #creating datasets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
    

    #Dataloaders for the datasets
    train_dataloader = DataLoader(train_set, batch_size = 64, shuffle = True)
    test_dataloader = DataLoader(test_set, batch_size = 64, shuffle = True)
    
    model.train()
    for i in range(epochs):
        print(i)
        train_loop(train_dataloader, model, loss_fn, optimizer)
    
    model.eval()
    model.to('cpu')

    model = torch.quantization.convert(model.eval(), inplace=False)
    model_accuracy = valid_loop(test_dataloader, model, loss_fn)

    return model, model_accuracy


In [23]:
model_accuracy = 0
while model_accuracy < 90:
    metric_list = []
    model_accuracies = []
    #for _ in range(5):
    result = main()
    model = result[0]
    model_accuracy = result[1]
    if model_accuracy < 90:
        continue
    model_accuracies.append(model_accuracy)
    stats_map = {}
    for key, value in classification_map.items():
        stats_map[key] = {
            "Specificity" : float(classification_map[key]["TN"]) / float(classification_map[key]["TN"] + classification_map[key]["FP"]),
            "Recall" : float(classification_map[key]["TP"]) / float(classification_map[key]["TP"] + classification_map[key]["FN"]),
            "Precision" : float(classification_map[key]["TP"]) / float(classification_map[key]["TP"] + classification_map[key]["FP"]),
            "Accuracy" : float(classification_map[key]["TP"] + classification_map[key]["TN"]) / float(classification_map[key]["TP"] + classification_map[key]["TN"] + classification_map[key]["FP"] + classification_map[key]["FN"])
        }
        stats_map[key]["F-score"] = 2.0 / float((1.0 / float(stats_map[key]["Precision"])) + (1.0 / float(stats_map[key]["Recall"])))
    metric_list.append(stats_map)
    for key, value in classification_map.items():
        classification_map[key] = {"TP" : 0,
                                "FP" : 0,
                                "TN" : 0,
                                "FN" : 0}


0


RuntimeError: Given groups=1, weight of size [32, 12, 3], expected input[64, 6, 250] to have 12 channels, but got 6 channels instead

In [18]:
for key, value in metric_list[0].items():
    if key == "FOL" or key =="FKL" or key == "BSC" or key =="SDL" or key =="Falls":
        specificity = mean([elem[key]["Specificity"] for elem in metric_list])
        recall = mean([elem[key]["Recall"] for elem in metric_list])
        precision = mean([elem[key]["Precision"] for elem in metric_list])
        fScore = mean([elem[key]["F-score"] for elem in metric_list])
        print("Specificity of " + key + ": " + str(specificity))
        print("Recall of " + key + ": " + str(recall))
        print("Precision of " + key + ": " + str(precision))
        print("F1-score of " + key + ": " + str(fScore))
        print("")
print("Model accuracy: " + str(mean(model_accuracies)))

Specificity of Falls: 0.9922077922077922
Recall of Falls: 0.9942857142857143
Precision of Falls: 0.9830508474576272
F1-score of Falls: 0.9886363636363635

Specificity of FOL: 0.9980806142034548
Recall of FOL: 0.8974358974358975
Precision of FOL: 0.9722222222222222
F1-score of FOL: 0.9333333333333331

Specificity of FKL: 0.9961464354527938
Recall of FKL: 0.8292682926829268
Precision of FKL: 0.9444444444444444
F1-score of FKL: 0.8831168831168832

Specificity of BSC: 0.9921104536489151
Recall of BSC: 0.9056603773584906
Precision of BSC: 0.9230769230769231
F1-score of BSC: 0.9142857142857143

Specificity of SDL: 0.9768339768339769
Recall of SDL: 0.9761904761904762
Precision of SDL: 0.7735849056603774
F1-score of SDL: 0.8631578947368421

Model accuracy: 93.92857142857143


In [14]:
from torch.utils.mobile_optimizer import optimize_for_mobile
def save_model(model):
    torchscript_model = torch.jit.script(model)
    torchscript_model_optimized = optimize_for_mobile(torchscript_model)
    torch.jit.save(torchscript_model_optimized, "MIS_RFD_model_quantized.pt")
    #torchscript_model_optimized._save_for_lite_interpreter("MIS_RFD_model_quantized.ptl")

In [15]:
save_model(model)